In [ ]:
import pandas as pd
import seaborn as sns
from haversine import haversine

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import sys
from datawand.parametrization import ParamHelper
ph = ParamHelper("../pipelines/EcmlTaxi.json",sys.argv)

# Haversine distance

**Conclusion:** I must use at least 3 digit in gps locations

### 2 digit precision : approx. 1 km

In [ ]:
print haversine((-8.62, 41.141412),(-8.61, 41.141412))
print haversine((-8.618643, 41.14),(-8.618643, 41.15))

### 3 digit precision: approx 100 m

In [ ]:
print haversine((-8.610, 41.141412),(-8.611, 41.141412))
print haversine((-8.618643, 41.150),(-8.618643, 41.151))

# Load Data

In [ ]:
data_folder = ph.get("data_folder")
part = ph.get("part")
lat_digits = ph.get("lat_digits")
lng_digits = ph.get("lng_digits")

In [ ]:
taxi_trips = pd.read_csv("%s/gen_train%s_lng%i_lat%i.csv" % (data_folder, part, lng_digits, lat_digits))

In [ ]:
taxi_trips_test = pd.read_csv("%s/gen_test_lng%i_lat%i.csv" % (data_folder, lng_digits, lat_digits))

In [ ]:
taxi_trips.head(1)

In [ ]:
taxi_trips.columns

# Time information

In [ ]:
taxi_trips.hist("DAY_OF_WEEK",bins=7)

In [ ]:
taxi_trips.hist("TIME_OF_DAY",bins=4)

from altair import *

Chart(taxi_trips).mark_point().encode(
    x='DAY_OF_WEEK',
    y='TIME_OF_DAY',
    size='count(TIME_OF_DAY)',
    color='count(TIME_OF_DAY)'
)

# Trip length

In [ ]:
taxi_trips_test["TRIP_SIZE"].mean()

In [ ]:
taxi_trips["TRIP_SIZE"].mean()

In [ ]:
taxi_trips_test["TRIP_LAT_UNIQUE"].mean()

In [ ]:
taxi_trips["TRIP_LAT_UNIQUE"].mean()

In [ ]:
taxi_trips_test["TRIP_LNG_UNIQUE"].mean()

In [ ]:
taxi_trips["TRIP_LNG_UNIQUE"].mean()

# Location information: Porto, Portugal

In [ ]:
taxi_trips_head = taxi_trips#.head(10000)
plt.scatter(taxi_trips_head["DESTINATION_LNG"], taxi_trips_head["DESTINATION_LAT"],c="b",alpha=0.6)
plt.scatter(taxi_trips_test["DESTINATION_LNG"], taxi_trips_test["DESTINATION_LAT"],c="r",alpha=0.6)

In [ ]:
plt.scatter(taxi_trips_head["DESTINATION_LNG_FULL"], taxi_trips_head["DESTINATION_LAT_FULL"],c="b",alpha=0.6)
plt.scatter(taxi_trips_test["DESTINATION_LNG_FULL"], taxi_trips_test["DESTINATION_LAT_FULL"],c="r",alpha=0.6)

In [ ]:
taxi_trips["MISSING_DATA"].value_counts()

taxi_trips["POLYLINE"] = taxi_trips["POLYLINE"].apply(lambda x: eval(x))

taxi_trips["POLYLINE_SIZE"] = taxi_trips["POLYLINE"].apply(lambda x: len(x))

Chart(taxi_trips).mark_point().encode(
    x='DAY_OF_WEEK',
    y='TIME_OF_DAY',
    size='mean(POLYLINE_SIZE)',
    color='mean(POLYLINE_SIZE)'
)

Chart(taxi_trips).mark_point().encode(
    x='DAY_OF_WEEK',
    y='TIME_OF_DAY',
    size='mean(POLYLINE_SIZE)',
    color='mean(POLYLINE_SIZE)'
)

Chart(taxi_trips).mark_point().encode(
    x='DAY_TYPE',
    y='TIME_OF_DAY',
    size='mean(POLYLINE_SIZE)',
    color='mean(POLYLINE_SIZE)'
)

#### Unique locations

gps_locs = {}
for i in xrange(len(taxi_trips)):
    for loc in set(taxi_trips.ix[i]["POLYLINE"]):
        if not loc in gps_locs:
            gps_locs[loc] = 0
        gps_locs[loc] += 1

len(gps_locs)